# FutBot - Preprocesamiento de datos

### Importacion de librerias

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import unicodedata
import pandas as pd
import seaborn as sns
import unicodedata

In [5]:
# Como el dataset original tiene muchas columnas, seteamos pandas para que se vean todas las columnas existentes en el dataset
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Importacion de datos

En caso de ya tener guardado jugadores_reducido.csv, ir a la seccion "Limpieza de datos - Liga Argentina"

In [ ]:
# Importacion de datos
csv_path = './data_cleaning/male_players.csv'
df = pd.read_csv(csv_path)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/male_players.csv'

Existen columnas que son propias del juego FIFA, y que para nuestro analisis no son utiles, por lo que decidimos ignorarlas en un nuevo dataset

In [ ]:
# Seleccion de columnas a utilizar en el proyecto
columnas_utiles = ['player_url', 'fifa_version', 'fifa_update','long_name', 'player_positions', 'dob',
                   'height_cm', 'weight_kg', 'league_name', 'league_level', 'club_name', 'club_position', 'club_jersey_number',
                   'nationality_name', 'nation_position', 'nation_jersey_number', 'preferred_foot']

df = df[columnas_utiles].copy() 

In [ ]:
df.head()

In [ ]:
# Guardamos el dataset reducido para no tener que cargar el original cada vez
df.to_csv("jugadores_reducido.csv", index=False)

### Limpieza de datos - Jugadores de la Liga Argentina

In [ ]:
# Ejecutar cuando se tiene el dataset reducido
df = pd.read_csv("jugadores_reducido.csv")
df.head()

In [ ]:
df['league_name'].dropna().unique()

1. Guardamos los jugadores cuyo nombre de liga sea "Liga Argentina"

In [ ]:
# Guardamos los jugadores de la liga profesional
liga_argentina = df[df['league_name'].str.lower().str.contains('liga profesional', na=False)]
liga_argentina.head()

In [ ]:
# Ejecutar en caso de querer tener guardada la liga argentina
liga_argentina.to_csv("liga_argentina.csv", index=False)

In [ ]:
# Ejecutar cuando se tiene guardada liga_argentina.csv
#liga_argentina = pd.read_csv("liga_argentina.csv")
#liga_argentina.head()

In [ ]:
liga_argentina.head()

In [ ]:
liga_argentina = liga_argentina.copy()

In [ ]:
# Total columnas y filas
liga_argentina.shape

2. Eliminacion de jugadores duplicados

In [ ]:
# Mantener la primera fila encontrada para cada long_name duplicado ya que las demas filas eran de versiones anteriores del jugador
liga_argentina = liga_argentina.drop_duplicates(subset=['long_name'], keep='first') 

In [ ]:
# Total columnas y filas sin duplicados
liga_argentina.shape

3. Eliminacion de features innecesarias

In [ ]:
# Eliminamos 'player_url', 'fifa_version', 'fifa_update' porque ya no se usan mas
liga_argentina.drop(columns=['player_url', 'fifa_version', 'fifa_update'], inplace=True)
liga_argentina.head()

In [ ]:
# Ejecutar en caso de querer guardar la liga argentina sin duplicados
liga_argentina.to_csv("liga_argentina_sin_duplicados.csv", index=False)

### Visualizaciones

1. Cantidad de jugadores por equipo dentro de la Liga Argentina

In [ ]:
# Gráfico que muestra la cantidad de jugadores por equipo

plt.figure(figsize=(15, 8)) # Aumenta el tamaño para mejor visualización
club_counts_all = liga_argentina['club_name'].value_counts() # Considera todos los clubes

sns.barplot(x=club_counts_all.index, y=club_counts_all.values, palette="viridis")

plt.title("Cantidad de jugadores por club (Todos los clubes)", fontsize=16)
plt.xlabel("Club", fontsize=12)
plt.ylabel("Cantidad de jugadores", fontsize=12)
plt.xticks(rotation=90, ha='right', fontsize=10) # Rota las etiquetas del eje X
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout() # Ajusta el diseño para evitar solapamiento
plt.show()

2. Cantidad de jugadores por pierna preferida

In [ ]:
# Gráfico que muestra la cantidad de jugadores por pierna preferida
plt.figure(figsize=(8, 6))
sns.countplot(x='preferred_foot', data=liga_argentina, palette='viridis')
plt.title('Cantidad de jugadores por pierna preferida', fontsize=16)
plt.xlabel('Pierna preferida', fontsize=12)
plt.ylabel('Cantidad de jugadores', fontsize=12)
plt.show()

3. Cantidad de jugadores por nacionalidad

In [ ]:
# Gráfico que muestra la cantidad de jugadores por nacionalidad
plt.figure(figsize=(14, 7))
nationality_counts = liga_argentina['nationality_name'].value_counts().head(20) # Top 20 nacionalidades

sns.barplot(x=nationality_counts.values, y=nationality_counts.index, palette='viridis')
plt.title('Top 20 Nacionalidades de jugadores en el dataset', fontsize=16)
plt.xlabel('Cantidad de jugadores', fontsize=12)
plt.ylabel('Nacionalidad', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# La mayoría de los jugadores son Argentinos, por ende podemos decir que este gráfico nos sirve para que, cuando entrenemos las preguntas, no usemos la pregunta "De qué nacionalidad es x jugador?" Porque va a tender a decir argentina (?)

### Tranformacion de datos

1. Transformacion de tipos de datos para un menor consumo de memoria

In [ ]:
liga_argentina.info()

In [ ]:
# Transformacion de tipos de datos
int1_cols = ['height_cm', 'weight_kg']
str_cols = ['long_name', 'player_positions']
datetime_cols = ['dob']
category_cols = ['league_name', 'club_name', 'club_position', 'nationality_name', 'nation_position', 'preferred_foot']
int_cols = ['league_level', 'club_jersey_number', 'nation_jersey_number']

# Convertir a int
for col in int1_cols:
    liga_argentina[col] = liga_argentina[col].astype('Int64')

# Convertir a string
for col in str_cols:
    liga_argentina[col] = liga_argentina[col].astype('str')

# Convertir a datetime
for col in datetime_cols:
    liga_argentina[col] = pd.to_datetime(liga_argentina[col], errors='coerce')

# Convertir a category
for col in category_cols:
    liga_argentina[col] = liga_argentina[col].astype('category')

# Convertir a Int64 (nullable integer)
for col in int_cols:
    liga_argentina[col] = liga_argentina[col].astype('Int64')

liga_argentina.info()

2. Analisis de nulos

In [ ]:
# Verificacion de nulos
liga_argentina.isnull().sum()

In [ ]:
liga_argentina[df['nation_position'].isnull()].head()

Observamos que los nulos provienen de nation_position y nation_jersey_number, y al analizar dichos casos entendimos que se tratan de jugadores que no se encuentran convocados en la seleccion de su pais. Para trabajar con ellos primero decidimos reemplazar las posiciones de los jugadores que se encuentran abreviadas en las originales o completas

3. Manejo de abreviaciones

In [ ]:
# Divide las posiciones de juego del player_position por coma y quita espacios (guardado en un array)
# Va a ser util para el diccionario de posiciones
liga_argentina['player_positions'] = liga_argentina['player_positions'].fillna('').str.split(', ')
liga_argentina.head()

In [ ]:
# Diccionario de abreviaciones a descripciones en español
posiciones_fifa = {
    "GK": "Arquero",
    "ST": "Delantero",
    "CF": "Delantero Centro",
    "LF": "Delantero Izquierdo",
    "RF": "Delantero Derecho",
    "LS": "Delantero Izquierdo",
    "RS": "Delantero Derecho",
    "LW": "Extremo Izquierdo",
    "RW": "Extremo Derecho",
    "CAM": "Mediocampista Ofensivo",
    "CM": "Mediocampista Central",
    "CDM": "Mediocampista Defensivo",
    "LAM": "Mediocampista Izquierdo Ofensivo",
    "RAM": "Mediocampista Derecho Ofensivo",
    "LCM": "Mediocentro Izquierdo",
    "RCM": "Mediocentro Derecho",
    "LM": "Mediocampista Izquierdo",
    "RM": "Mediocampista Derecho",
    "LDM": "Mediocampista Defensivo Izquierdo",
    "RDM": "Mediocampista Defensivo Derecho",
    "LB": "Lateral Izquierdo",
    "RB": "Lateral Derecho",
    "LCB": "Defensor Central Izquierdo",
    "RCB": "Defensor Central Derecho",
    "CB": "Defensor Central",
    "LWB": "Carrilero Izquierdo",
    "RWB": "Carrilero Derecho",
    "SUB": "Suplente",
    "RES": "Reserva",
    "No Convocado": "No Convocado"
}

In [ ]:
# Crea una nueva columna con las descripciones traducidas de players_positions
liga_argentina['player_positions_desc'] = liga_argentina['player_positions'].apply(
    lambda posiciones: [posiciones_fifa.get(pos.strip(), f"Desconocida ({pos})") for pos in posiciones]
)
liga_argentina.head()

In [ ]:
# Crea una nueva columna con las descripciones traducidas de club_position
liga_argentina['club_position_desc'] = liga_argentina['club_position'].astype(str).apply(
    lambda x: posiciones_fifa.get(x, f'Desconocida ({x})')
)
liga_argentina.head()

Imputacion de nulos en nation_position

In [ ]:
# Agregar categoría "No Convocado" a la columna nation_position para los que no se encuentran convocados en la seleccion
liga_argentina['nation_position'] = liga_argentina['nation_position'].cat.add_categories(['No Convocado'])

# Reemplazamos NaN por "No Convocado"
liga_argentina['nation_position'] = liga_argentina['nation_position'].fillna('No Convocado')

In [ ]:
# Crea una nueva columna con las descripciones traducidas de nation_position
liga_argentina['nation_position_desc'] = liga_argentina['nation_position'].astype(str).apply(
    lambda x: posiciones_fifa.get(x, f'Desconocida ({x})')
)
liga_argentina.head()

Imputacion de nulos en nation_jersey_number

In [ ]:
# Imputamos un numero alto para identificar que no esta convocado el jugador a la seleccion de su pais
liga_argentina['nation_jersey_number'] = liga_argentina['nation_jersey_number'].fillna(99)
liga_argentina.head()

4. Eliminacion de features innecesarias

In [ ]:
# Eliminamos las columnas repetidas en inglés para quedarnos con sus descripciones
liga_argentina.drop(columns=['player_positions', 'club_position', 'nation_position'], inplace=True)

5. Traduccion de valores ENG-ESP

In [ ]:
# Diccionario de traducción
foot_trad = {
    'Right': 'Derecha',
    'Left': 'Izquierda'
}

# Aplicar la traducción
liga_argentina['preferred_foot'] = liga_argentina['preferred_foot'].map(foot_trad)
liga_argentina.head()

6. Ultimos detalles

In [ ]:
# Se necesita eliminar los corchetes de las posiciones de juego
def limpiar_corchetes(text):
    if isinstance(text, list):
        return ", ".join(text)
    if isinstance(text, str) and text.startswith("[") and text.endswith("]"):
        return text.strip("[]").replace("'", "").replace('"', '')
    return text

In [ ]:
# Aplicamos la funcion de limpiar corchetes a la columna player_positions_
liga_argentina['player_positions_desc'] = liga_argentina['player_positions_desc'].apply(limpiar_corchetes)
liga_argentina.head()

### Normalizacion

In [ ]:
# Normalización de datos

def normalizar_texto(texto):
    if isinstance(texto, list):
        # Normaliza cada posición en la lista
        return [normalizar_texto(pos) for pos in texto]
    elif pd.isna(texto) or texto == "":
        # Retorna una lista vacía o maneja de alguna otra forma los NaN/vacíos si es necesario
        return []
    elif pd.isna(texto):
        return ""
    texto = str(texto).lower()  # para pasarlo a minúsculas
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode('utf-8')  # para descomponer caracteres especiales y borrar tildes
    return texto.strip()  # borrar espacios que estén al comienzo y al final

cols_a_normalizar = [
    'long_name',
    'league_name',
    'club_name',
    'nationality_name',
    'preferred_foot',
    'player_positions_desc',
    'club_position_desc',
    'nation_position_desc'
]

for col in cols_a_normalizar:
    liga_argentina[col] = liga_argentina[col].apply(normalizar_texto)
liga_argentina.head()

In [ ]:
liga_argentina.to_csv("liga_argentina_model.csv", index=False)

### Visualizaciones Finales

In [ ]:
# Gráfico para ver la cantidad de jugadores por posición
# Preparar datos para el gráfico de posiciones de juego
# Primero, expandimos la columna player_positions_trans para tener una fila por cada posición
posiciones_expandidas = liga_argentina['player_positions_desc'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True).to_frame('posicion_expandida')

# Unir con el DataFrame original para obtener las filas completas
df_posiciones_expandidas = liga_argentina.drop('player_positions_desc', axis=1).join(posiciones_expandidas)

# Contar la frecuencia de cada posición
posicion_counts = df_posiciones_expandidas['posicion_expandida'].value_counts()

# Diccionario para agrupar posiciones (simplificación)
grupos_posiciones = {
    'Arquero': ['arquero'],
    'Defensa': ['lateral izquierdo', 'lateral derecho', 'defensor central izquierdo', 'defensor central derecho', 'defensor central', 'carrilero izquierdo', 'carrilero derecho'],
    'Mediocampo': ['mediocampista ofensivo', 'mediocampista central', 'mediocampista defensivo', 'mediocampista izquierdo ofensivo', 'mediocampista derecho ofensivo', 'mediocentro izquierdo', 'mediocentro derecho', 'mediocampista izquierdo', 'mediocampista derecho', 'mediocampista defensivo izquierdo', 'mediocampista defensivo derecho'],
    'Delantero': ['delantero', 'delantero centro', 'delantero izquierdo', 'delantero derecho', 'extremo izquierdo', 'extremo derecho'],
    'Otros': ['suplente', 'reserva', 'no convocado', 'desconocida (nan)']
}

# Crear una nueva columna con las posiciones agrupadas
def agrupar_posicion(posicion):
    for grupo, posiciones_list in grupos_posiciones.items():
        if posicion in posiciones_list:
            return grupo
    return 'Otros' # Por si acaso alguna posición no está mapeada

df_posiciones_expandidas['grupo_posicion'] = df_posiciones_expandidas['posicion_expandida'].apply(agrupar_posicion)

# Contar la frecuencia de las posiciones agrupadas
grupo_posicion_counts = df_posiciones_expandidas['grupo_posicion'].value_counts()

# Gráfico de barras de las posiciones agrupadas
plt.figure(figsize=(10, 7))
sns.barplot(x=grupo_posicion_counts.index, y=grupo_posicion_counts.values, palette="viridis")
plt.title('Distribución de Grupos de Posiciones de Juego', fontsize=16)
plt.xlabel('Grupo de Posición', fontsize=12)
plt.ylabel('Cantidad de Jugadores', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Opcional: Gráfico de barras de las posiciones más detalladas (Top N)
# Puedes ajustar N para ver más o menos posiciones
n_top_positions = 20
posiciones_top = posicion_counts.head(n_top_positions)

plt.figure(figsize=(12, 8))
sns.barplot(x=posiciones_top.values, y=posiciones_top.index, palette="viridis")
plt.title(f'Top {n_top_positions} Posiciones de Juego más Frecuentes', fontsize=16)
plt.xlabel('Cantidad de Jugadores', fontsize=12)
plt.ylabel('Posición', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()